In [ ]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

from data import get_dataloaders
from model import Dehazer

In [ ]:
%load_ext autoreload
%autoreload 2

random.seed(43)
torch.manual_seed(43)

In [ ]:
# options: ["ohaze", "dh/Middlebury", "dh/NYU"]
DATASET = "dh/NYU"
BATCH_SIZE = 8
TRAIN_SPLIT = 0.8
train_loader, test_loader = get_dataloaders(DATASET, TRAIN_SPLIT, BATCH_SIZE, tmap_gt=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = Dehazer(in_channels=3, out_channels=1)
num_params_m = sum(torch.numel(p) for p in model.parameters())
print(f"Number of parameters in the model: {num_params_m}")

criterion = nn.MSELoss()

learning_rate = 2e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def test(model, test_loader, writer, epoch):
    model.eval()

    with torch.no_grad():
        for images, targets in test_loader:
            images = images.to(device)

            # Forward pass
            outputs = model(images)

            # Convert the outputs and targets to numpy arrays
            outputs_np = outputs.cpu().numpy()
            targets_np = targets.numpy()

            num_outs = outputs.shape[0]
            fig, axs = plt.subplots(2, num_outs, figsize=(20, 8))

            for i in range(num_outs):
                axs[0, i].imshow(targets_np[i].transpose(1, 2, 0), cmap='gray')
                axs[0, i].set_title("Ground Truth")
                axs[0, i].axis("off")

                # Plot predicted image in the second row
                axs[1, i].imshow(outputs_np[i].transpose(1, 2, 0), cmap='gray')
                axs[1, i].set_title("Output")
                axs[1, i].axis("off")

            plt.tight_layout()
            plt.savefig(f'plot_{epoch}.png')

            # Load the image file as a tensor
            image = torch.from_numpy(np.array(Image.open('plot.png'))).permute(2, 0, 1).float() / 255

            # Add the image to the SummaryWriter
            writer.add_image('Plot', image)
            plt.show()
            break

    writer.close()


In [ ]:
def train(model, train_loader, test_loader, num_epochs):
    model.to(device)

    log_dir = f"./logs/{DATASET}/lr{learning_rate}_epochs{num_epochs}"
    writer = SummaryWriter(log_dir)
    weights_dir = os.path.join(log_dir, "weights")
    os.makedirs(weights_dir, exist_ok=True)

    for epoch in range(num_epochs):

            running_loss = 0.0
            for idx, (images, targets) in enumerate(train_loader):
                images = images.to(device)
                targets = targets.to(device)

                # Forward pass
                outputs = model(images)

                # Compute the loss
                loss = criterion(outputs, targets)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            epoch_loss = running_loss / len(train_loader)
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.6f}")

            if (epoch+1)%3 == 0:
                test(model, test_loader, writer, epoch)
                model.train()
                checkpoint = {
                        'epoch' : epoch + 1,
                        'state_dict': model.state_dict(),
                        'optimizer': optimizer.state_dict()
                    }
                torch.save(checkpoint, os.path.join(weights_dir, f"{(epoch+1):03}.pth"))

    writer.close()

In [ ]:
NUM_EPOCHS = 30
train(model, train_loader, test_loader, NUM_EPOCHS)